In [124]:
# !pip install config
!pip install geoparquet 
import geoparquet as gpq
import pickle
# import config
import os
import gc

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rasterio as rio

from rasterio.io import MemoryFile

from rasterio.features import rasterize
# from utils import parquet_to_gdf
# from Grid_to_parquet import get_dataset

TEMPLATE_BLOB_NAME = "nwm.20221001/forcing_medium_range/nwm.t00z.medium_range.forcing.f001.conus.nc"



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [125]:
NWM_BUCKET = "national-water-model"

# WKT strings extracted from NWM grids
CONUS_NWM_WKT = 'PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DATUM["D_Sphere",SPHEROID["Sphere",6370000.0,0.0]], \
PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",0.0],\
PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-97.0],PARAMETER["standard_parallel_1",30.0],\
PARAMETER["standard_parallel_2",60.0],PARAMETER["latitude_of_origin",40.0],UNIT["Meter",1.0]]'

HI_NWM_WKT = 'PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DATUM["D_Sphere",SPHEROID["Sphere",6370000.0,0.0]],\
PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",0.0],\
PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-157.42],PARAMETER["standard_parallel_1",10.0],\
PARAMETER["standard_parallel_2",30.0],PARAMETER["latitude_of_origin",20.6],UNIT["Meter",1.0]]'

PR_NWM_WKT = 'PROJCS["Sphere_Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DATUM["D_Sphere",SPHEROID["Sphere",6370000.0,0.0]],\
PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["false_easting",0.0],\
PARAMETER["false_northing",0.0],PARAMETER["central_meridian",-65.91],PARAMETER["standard_parallel_1",18.1],\
PARAMETER["standard_parallel_2",18.1],PARAMETER["latitude_of_origin",18.1],UNIT["Meter",1.0]]'

In [29]:
import os
from pathlib import Path


CACHE_DIR = Path(".", "data_3", "cache")
NWM_CACHE_DIR = os.path.join(CACHE_DIR, "nwm")
USGS_CACHE_DIR = os.path.join(CACHE_DIR, "usgs")
GEO_CACHE_DIR = os.path.join(CACHE_DIR, "geo")

NWM_CACHE_H5 = os.path.join(NWM_CACHE_DIR, "gcp_client.h5")

PARQUET_CACHE_DIR = os.path.join(CACHE_DIR, "parquet")
MEDIUM_RANGE_FORCING_PARQUET = os.path.join(PARQUET_CACHE_DIR, "forcing_medium_range")
FORCING_ANALYSIS_ASSIM_PARQUET = os.path.join(PARQUET_CACHE_DIR, "forcing_analysis_assim")
MEDIUM_RANGE_PARQUET = os.path.join(PARQUET_CACHE_DIR, "medium_range")
USGS_PARQUET = os.path.join(PARQUET_CACHE_DIR, "usgs")

HUC10_SHP_FILEPATH = os.path.join(GEO_CACHE_DIR, "wbdhu10_conus.shp")
HUC10_PARQUET_FILEPATH = os.path.join(GEO_CACHE_DIR, "wbdhu10_conus.parquet")
HUC10_MEDIUM_RANGE_WEIGHTS_FILEPATH = os.path.join(GEO_CACHE_DIR, "wbdhu10_medium_range_weights.pkl")

ROUTE_LINK_FILE = os.path.join(NWM_CACHE_DIR, "RouteLink_CONUS.nc")
ROUTE_LINK_PARQUET = os.path.join(NWM_CACHE_DIR, "route_link_conus.parquet")

In [30]:
def parquet_to_gdf(parquet_filepath: str) -> gpd.GeoDataFrame:
    gdf = gpd.read_parquet(parquet_filepath)
    return gdf

def get_cache_dir(create: bool = True):
    if not os.path.exists(NWM_CACHE_DIR) and create:
        os.mkdir(NWM_CACHE_DIR)
    if not os.path.exists(NWM_CACHE_DIR):
        raise NotADirectoryError
    return NWM_CACHE_DIR


def make_parent_dir(filepath):
    Path(filepath).parent.mkdir(parents=True, exist_ok=True)



In [31]:
def get_dataset(
        blob_name: str,
        use_cache: bool = True
) -> xr.Dataset:
    """Retrieve a blob from the data service as xarray.Dataset.
    Based largely on OWP HydroTools.
    Parameters
    ----------
    blob_name: str, required
        Name of blob to retrieve.
    use_cacahe: bool, default True
        If cache should be used.  
        If True, checks to see if file is in cache, and 
        if fetched from remote will save to cache.
    Returns
    -------
    ds : xarray.Dataset
        The data stored in the blob.
    """
    nc_filepath = os.path.join(get_cache_dir(), blob_name)
    make_parent_dir(nc_filepath)

    # If the file exists and use_cache = True
    if os.path.exists(nc_filepath) and use_cache:
        # Get dataset from cache
        ds = xr.load_dataset(
            nc_filepath,
            engine='h5netcdf',
        )
        return ds
    else:
        # Get raw bytes
        raw_bytes = get_blob(blob_name)
        # Create Dataset
        ds = xr.load_dataset(
            MemoryFile(raw_bytes),
            engine='h5netcdf',
        )
        if use_cache:
            # Subset and cache
            ds["RAINRATE"].to_netcdf(
                nc_filepath,
                engine='h5netcdf',
            )
        return ds




In [46]:

def generate_weights_file(
    gdf: gpd.GeoDataFrame,
    src: xr.DataArray,
    weights_filepath: str,
    crosswalk_dict_key: str, 
):
    """Generate a weights file."""
    
    gdf_proj = gdf.to_crs(CONUS_NWM_WKT)
    
    crosswalk_dict = {}
    
    # This is a probably a really poor performing way to do this 
    for index, row in gdf_proj.iterrows():
        geom_rasterize = rasterize([(row["geometry"], 1)],
                                   out_shape=src.rio.shape,
                                   transform=src.rio.transform(),
                                   all_touched=True,
                                   fill=0,
                                   dtype='uint8')
        if crosswalk_dict_key:
            crosswalk_dict[row[crosswalk_dict_key]
                           ] = np.where(geom_rasterize == 1)
        else:
            crosswalk_dict[index] = np.where(geom_rasterize == 1)

    with open(weights_filepath, 'wb') as f:
        pickle.dump(crosswalk_dict, f)

        

In [48]:
def add_zonalstats_to_gdf_weights(
    gdf: gpd.GeoDataFrame,
    src: xr.DataArray,
    weights_filepath: str,
) -> gpd.GeoDataFrame:
    """Calculates zonal stats and adds to GeoDataFrame"""

    df = calc_zonal_stats_weights(src, weights_filepath)
    gdf_map = gdf.merge(df, left_on="huc10", right_on="catchment_id")

    return gdf_map

In [49]:
from google.cloud import storage
def get_blob(
    blob_name: str,
    bucket: str = NWM_BUCKET
) -> bytes:
    """Retrieve a blob from the data service as bytes.
    Based largely on OWP HydroTools.
    Parameters
    ----------
    blob_name : str, required
        Name of blob to retrieve.
    Returns
    -------
    data : bytes
        The data stored in the blob.
    """
    # Setup anonymous client and retrieve blob data
    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket)
    return bucket.blob(blob_name).download_as_bytes(timeout=120)



In [52]:
%%time
### MAIN
ds = get_dataset(TEMPLATE_BLOB_NAME, use_cache=True)

CPU times: user 305 ms, sys: 46.5 ms, total: 351 ms
Wall time: 356 ms


In [53]:
%%time
polygonfile = gpd.read_file("03w/nextgen_03W.gpkg",layer="divides")

CPU times: user 1.56 s, sys: 52.4 ms, total: 1.62 s
Wall time: 1.65 s


In [60]:
%%time
polygonfile.to_parquet("03w/ng_03W.parquet")

CPU times: user 171 ms, sys: 21.4 ms, total: 193 ms
Wall time: 209 ms


In [63]:
%%time
gdf = parquet_to_gdf("03w/ng_03W.parquet")

CPU times: user 271 ms, sys: 53.4 ms, total: 324 ms
Wall time: 340 ms


In [64]:
%%time
src = ds["RAINRATE"]
generate_weights_file(gdf, src, "data_3/weights.pkl", crosswalk_dict_key="id")

CPU times: user 27min 19s, sys: 1min 20s, total: 28min 39s
Wall time: 36min 34s


In [134]:
# !pip install ipdb
def calc_zonal_stats_weights(
    src: xr.DataArray,
    weights_filepath: str,
) -> pd.DataFrame:
    """Calculates zonal stats"""

    # Open weights dict from pickle
    # This could probably be done once and passed as a reference.
    with open(weights_filepath, 'rb') as f:
        crosswalk_dict = pickle.load(f)

    r_array = src.values[0]
    r_array[r_array == src.rio.nodata] = np.nan

    mean_dict = {}
    for key, value in crosswalk_dict.items():
        mean_dict[key] = np.nanmean(r_array[value])

    df = pd.DataFrame.from_dict(mean_dict,
                                orient='index',
                                columns=['value'])

    df.reset_index(inplace=True, names="catchment_id")

    # This should not be needed, but without memory usage grows
    del crosswalk_dict
    del f
    gc.collect()

    return df

In [135]:
%%time
calc_zonal_stats_weights(src, "data_3/weights.pkl")

CPU times: user 752 ms, sys: 13.8 ms, total: 765 ms
Wall time: 767 ms


,catchment_id,value
0,cat-107068,0.0
1,cat-107233,0.0
2,cat-107683,0.0
3,cat-108167,0.0
4,cat-112977,0.0
...,...,...
30867,cat-137554,0.0
30868,cat-106885,0.0
30869,cat-137071,0.0
30870,cat-137068,0.0


In [140]:
def get_forcing_dict_RTIway(
    pickle_file, # This would be a Feature list for parallel calling -- 
    # if there is a stored weights file, we use it 
    # (checking for an optional flag to force re-creation of the weights...)
    folder_prefix,
    file_list,
):

    var = "RAINRATE"
    reng = "rasterio"
    filehandles = [xr.open_dataset(folder_prefix / f, engine=reng)[var] for f in file_list]
    # filehandles = [get_dataset("data/" + f, use_cache=True) for f in file_list]
    stats = []

    for _i, f in enumerate(filehandles):
        print(f"{_i}, {round(_i/len(file_list), 2)*100}".ljust(40), end="\r")
        stats.append(calc_zonal_stats_weights(f, pickle_file))

    [f.close() for f in filehandles]
    return stats



In [174]:

def get_forcing_dict_RTIway2(
    pickle_file, # This would be a Feature list for parallel calling -- 
    # if there is a stored weights file, we use it 
    # (checking for an optional flag to force re-creation of the weights...)
    gpkg_divides,
    folder_prefix,
    filelist,
    var_list,
):
    reng = "rasterio"
    pick_val = "value"

    df_dict = {}
    for _v in var_list:
        df_dict[_v] = pd.DataFrame(index=gpkg_divides.index)

    ds_list = []
    for _i, _nc_file in enumerate(filelist):
        # _nc_file = ("nwm.t00z.medium_range.forcing.f001.conus.nc")
        _full_nc_file = folder_prefix.joinpath(_nc_file)

        with xr.open_dataset(_full_nc_file, engine=reng) as _xds:
        #_xds = ds_list[_i]
            print(f"{_i}, {round(_i/len(filelist), 5)*100}".ljust(40), end="\r")
            for _v in var_list:
                _src = _xds[_v]

                # import ipdb; ipdb.set_trace()
                _df_zonal_stats = (calc_zonal_stats_weights(_src, pickle_file))
                # if adding statistics back to original GeoDataFrame
                # gdf3 = pd.concat([gpkg_divides, _df_zonal_stats], axis=1)
                df_dict[_v][_xds.time.values[0]] = _df_zonal_stats[pick_val]

    [_xds.close() for _xds in ds_list]

    return df_dict



In [ ]:
%%time
import time
folder_prefix = Path("data")
list_of_files = [
    f"nwm.t12z.medium_range.forcing.f{_r:03}.conus.nc" for _r in range(1, 241)
]

f_03 = "03w/nextgen_03W.gpkg"
gpkg_divides = gpd.read_file(f_03, layer="divides")
var_list = [
    "U2D",
    "V2D",
    "LWDOWN",
    "RAINRATE",
    "T2D",
    "Q2D",
    "PSFC",
    "SWDOWN",
]


# file_list = list_of_files[0:18]
file_list = list_of_files[0:3]
file_list = list_of_files[0:]

pickle_file = "data_3/weights.pkl"

start_time = time.time()
print(f"Working on the new way")
fd2 = get_forcing_dict_RTIway2(
    pickle_file,
    gpkg_divides,
    folder_prefix,
    file_list,
    var_list,
)
print(time.time() - start_time)



Working on the new way


In [176]:
fd2

{'U2D':        2022-08-24 13:00:00  2022-08-24 14:00:00  2022-08-24 15:00:00
 0                 0.093212            -0.436970            -0.097939
 1                -0.008313            -0.587000            -0.435375
 2                -0.442512            -0.937268            -0.507317
 3                -0.178103            -0.600897            -0.614621
 4                -0.694703            -1.009649            -1.231270
 ...                    ...                  ...                  ...
 30867            -1.230500            -1.497500            -1.447500
 30868            -0.728547            -0.795235            -0.682782
 30869             0.348000             0.298625             0.092250
 30870             0.331000             0.297000             0.093000
 30871            -0.856077            -1.326692            -1.195231
 
 [30872 rows x 3 columns],
 'V2D':        2022-08-24 13:00:00  2022-08-24 14:00:00  2022-08-24 15:00:00
 0                -2.467212            -2.61878

In [ ]:
# %%time
# if 1 == 1:
#     _f = folder_prefix / list_of_files[0]
#     src = xr.open_dataset(_f)["RAINRATE"]
#     generate_weights_file(gdf, src, "data_3/weights_alt.pkl", crosswalk_dict_key="id")
    

In [149]:
len(fd2)

240

In [151]:
fd2[0]


,catchment_id,value
0,cat-107068,0.000000e+00
1,cat-107233,5.745148e-07
2,cat-107683,3.051579e-07
3,cat-108167,0.000000e+00
4,cat-112977,0.000000e+00
...,...,...
30867,cat-137554,0.000000e+00
30868,cat-106885,0.000000e+00
30869,cat-137071,0.000000e+00
30870,cat-137068,0.000000e+00


In [ ]:
import pandas as pd

pcp_var = pd.read_csv("RAINRATE.csv", index_col=0)#.rename("APCP_surface")
lw_var = pd.read_csv("LWDOWN.csv", index_col=0)#.rename("DLWRF_surface")
sw_var = pd.read_csv("SWDOWN.csv", index_col=0)#.rename("DSWRF_surface")
sp_var = pd.read_csv("PSFC.csv", index_col=0)#.rename("SPFH_2maboveground")
tmp_var = pd.read_csv("T2D.csv", index_col=0)#.rename("TMP_2maboveground")
u2d_var = pd.read_csv("U2D.csv", index_col=0)#.rename("UGRD_10maboveground")
v2d_var = pd.read_csv("V2D.csv", index_col=0)#.rename("VGRD_10maboveground")
pcp_var2 = pd.read_csv("RAINRATE.csv", index_col=0)#.rename("precip_rate") ##BROKEN!!

for _i in range(0,40000):
    #_i = 0
    try:
        pcp_var_0 = pcp_var.transpose()[_i].rename("APCP_surface")
        lw_var_0 = lw_var.transpose()[_i].rename("DLWRF_surface")
        sw_var_0 = sw_var.transpose()[_i].rename("DSWRF_surface")
        sp_var_0 = sp_var.transpose()[_i].rename("SPFH_2maboveground")
        tmp_var_0 = tmp_var.transpose()[_i].rename("TMP_2maboveground")
        u2d_var_0 = u2d_var.transpose()[_i].rename("UGRD_10maboveground")
        v2d_var_0 = v2d_var.transpose()[_i].rename("VGRD_10maboveground")
        pcp_var2_0 = pcp_var2.transpose()[_i].rename("precip_rate") ##BROKEN!!

        d = pd.concat([pcp_var_0, lw_var_0, sw_var_0, sp_var_0, tmp_var_0, u2d_var_0, v2d_var_0, pcp_var2_0], axis=1)
        d.index.name = "time"


        d.to_csv(f"input_data/cat03w_{_i:07}.csv")
    except:
        print(f"no data for watershed {_i}", end="\t")